# K-fold cross validation
데이터 셋을 여러 개로 나누어 하나씩 테스트 셋으로 사용하고, 나머지를 모두 합해서 학습 셋으로 사용하는 방법이다. 이렇게 하면 가지고 있는 데이터의 100%를 테스트 셋으로 사용할 수 있고, 데이터가 충분하지 않아도 좋은 결과를 낼 수 있다.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install tensorflow==2.0

  Using cached https://files.pythonhosted.org/packages/46/0f/7bd55361168bb32796b360ad15a25de6966c9c1beb58a8e30c01c8279862/tensorflow-2.0.0-cp36-cp36m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/76/54/99b9d5d52d5cb732f099baaaf7740403e83fe6b0cedde940fabd2b13d75a/tensorboard-2.0.2-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/fc/08/8b927337b7019c374719145d1dceba21a8bb909b93b1ad6f8fb7d22c1ca1/tensorflow_estimator-2.0.1-py2.py3-none-any.whl
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow~=2.1.0, but you'll have tensorflow 2.0.0 which is incompatible.
ERROR: tensorflow-federated 0.12.0 has requirement tensorflow-addons~=0.7.0, but you'll have tensorflow-addons 0.8.3 which is incompatible.
  Found existing installation: tensorboard 2.1.1
    Uninstalling tensorboard-2.1.1:
      Successfully uninstalled tensorboard-2.1.1
  Found existing installation: tensorflow-estimator 2.1.0
    Uninstalling tensorflow-estimator-2.

In [3]:
# TF 2.0 must be installed manually as the default version of TF in Colab is yet 1.15.0.
import tensorflow as tf
print(tf.__version__)

2.0.0


In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

import numpy as np
import pandas as pd

In [0]:
seed = 0
np.random.seed(seed)
tf.random.set_seed(3)

In [6]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/deep-learning-for-everyone/dataset/sonar.csv', header=None)
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,0.1609,0.1582,0.2238,0.0645,0.0660,0.2273,0.3100,0.2999,0.5078,0.4797,0.5783,0.5071,0.4328,0.5550,0.6711,0.6415,0.7104,0.8080,0.6791,0.3857,0.1307,0.2604,0.5121,0.7547,0.8537,0.8507,0.6692,0.6097,0.4943,0.2744,0.0510,0.2834,0.2825,0.4256,0.2641,0.1386,0.1051,0.1343,0.0383,0.0324,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,0.4918,0.6552,0.6919,0.7797,0.7464,0.9444,1.0000,0.8874,0.8024,0.7818,0.5212,0.4052,0.3957,0.3914,0.3250,0.3200,0.3271,0.2767,0.4423,0.2028,0.3788,0.2947,0.1984,0.2341,0.1306,0.4182,0.3835,0.1057,0.1840,0.1970,0.1674,0.0583,0.1401,0.1628,0.0621,0.0203,0.0530,0.0742,0.0409,0.0061,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,0.6333,0.7060,0.5544,0.5320,0.6479,0.6931,0.6759,0.7551,0.8929,0.8619,0.7974,0.6737,0.4293,0.3648,0.5331,0.2413,0.5070,0.8533,0.6036,0.8514,0.8512,0.5045,0.1862,0.2709,0.4232,0.3043,0.6116,0.6756,0.5375,0.4719,0.4647,0.2587,0.2129,0.2222,0.2111,0.0176,0.1348,0.0744,0.0130,0.0106,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,0.0881,0.1992,0.0184,0.2261,0.1729,0.2131,0.0693,0.2281,0.4060,0.3973,0.2741,0.3690,0.5556,0.4846,0.3140,0.5334,0.5256,0.2520,0.2090,0.3559,0.6260,0.7340,0.6120,0.3497,0.3953,0.3012,0.5408,0.8814,0.9857,0.9167,0.6121,0.5006,0.3210,0.3202,0.4295,0.3654,0.2655,0.1576,0.0681,0.0294,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,0.4152,0.3952,0.4256,0.4135,0.4528,0.5326,0.7306,0.6193,0.2032,0.4636,0.4148,0.4292,0.5730,0.5399,0.3161,0.2285,0.6995,1.0000,0.7262,0.4724,0.5103,0.5459,0.2881,0.0981,0.1951,0.4181,0.4604,0.3217,0.2828,0.2430,0.1979,0.2444,0.1847,0.0841,0.0692,0.0528,0.0357,0.0085,0.0230,0.0046,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R


In [0]:
dataset = df.values
X = dataset[:,0:60].astype(float)
Y_obj = dataset[:,60]

In [0]:
# 문자열 변환
e = LabelEncoder()
e.fit(Y_obj)
Y = e.transform(Y_obj)

**K-fold cross validation**

In [9]:
# 10개의 파일로 나눔.
# StratifiedKFold(): KFold()에 비해 stratified 는 label 의 분포를 유지시켜준다. 
# 즉, 각 fold 안의 데이터셋의 label 분포가 전체 데이터셋의 label 분포를 따른다. (각 fold가 전체 데이터셋을 잘 대표)
# 모델을 학습시킬 때 편향되지 않게 학습시킬 수 있다.

n_fold = 10
skf = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

for train_index, test_index in skf.split(X, Y): 
    print("TRAIN:", train_index, "TEST:", test_index)

TRAIN: [  0   1   3   4   5   6   7   8   9  10  11  12  13  15  16  17  18  20
  21  23  24  25  27  28  29  30  31  32  33  34  35  36  37  38  39  41
  42  43  45  46  47  48  50  51  53  54  55  56  57  58  59  60  61  62
  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80
  81  82  83  84  85  86  87  88  89  90  92  93  94  95  96  97  98  99
 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 118 120
 121 123 124 125 126 127 129 130 131 132 133 134 135 136 137 138 139 140
 142 143 144 145 146 147 148 149 150 152 153 154 155 156 157 158 159 160
 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178
 179 180 181 182 183 184 185 186 187 188 189 190 192 193 194 195 198 199
 200 201 202 203 205 206 207] TEST: [  2  14  19  22  26  40  44  49  52  91 100 117 119 122 128 141 151 191
 196 197 204]
TRAIN: [  0   1   2   3   4   5   7   8   9  10  11  12  14  15  16  17  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34

In [10]:
# 빈 accuracy 배열
accuracy = []

# 모델을 만들고 실행하는 부분을 반복
for train, test in skf.split(X, Y):
    # 모델 설정
    model = Sequential()
    model.add(Dense(24, input_dim=60, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    # 모델 컴파일
    model.compile(loss='mean_squared_error',
                  optimizer='adam',
                  metrics=['accuracy'])
    # 모델 실행
    model.fit(X[train], Y[train], epochs=100, batch_size=5)
    k_accuracy = "%.4f" % (model.evaluate(X[test], Y[test])[1])
    accuracy.append(k_accuracy)

# 결과 출력
print("\n %.f fold accuracy:" % n_fold, accuracy)

Train on 187 samples
Epoch 1/100
187/187 [==============================] - 1s 3ms/sample - loss: 0.2442 - accuracy: 0.5348
Epoch 2/100
187/187 [==============================] - 0s 331us/sample - loss: 0.2254 - accuracy: 0.6578
Epoch 3/100
187/187 [==============================] - 0s 319us/sample - loss: 0.2114 - accuracy: 0.6952
Epoch 4/100
187/187 [==============================] - 0s 365us/sample - loss: 0.1942 - accuracy: 0.7166
Epoch 5/100
187/187 [==============================] - 0s 307us/sample - loss: 0.1806 - accuracy: 0.7754
Epoch 6/100
187/187 [==============================] - 0s 345us/sample - loss: 0.1671 - accuracy: 0.8075
Epoch 7/100
187/187 [==============================] - 0s 360us/sample - loss: 0.1570 - accuracy: 0.7861
Epoch 8/100
187/187 [==============================] - 0s 337us/sample - loss: 0.1498 - accuracy: 0.8128
Epoch 9/100
187/187 [==============================] - 0s 327us/sample - loss: 0.1443 - accuracy: 0.8182
Epoch 10/100
187/187 [==============